# Working with JSON Data in the CDCS

<p class="lead">This notebook provides a step-by-step guide for handling JSON schemas and datasets, running queries, and displaying results. By the end of this tutorial, you will be proficient in managing both basic and complex JSON structures within the CDCS platform.</p>

<h2 class="text-primary">Step 1: Accessing the CDCS Platform</h2>

<div class="alert alert-info">
  Ensure that your CDCS environment is up and running. Once the environment is active, you can start testing and exploring.
</div>

<p class="lead"><strong>base_url</strong>: This variable defines the local CDCS platform URL, serving as a prefix for all API calls and platform access.</p>

<div class="alert alert-warning">
    <strong>Note:</strong> Ensure the URL does not end with a '/' to avoid errors.
</div>


In [ ]:
# Base URL for the service
base_url = input("Enter your CDCS url (eg: 'http://127.0.0.1:8000'): ")

<p class="lead">To access the platform:</p>
<ol class="list-group lead">
    <li class="list-group-item">Open your web browser.</li>
    <li class="list-group-item">Navigate to your CDCS app using the url entered</a>.</li>
</ol>

<div class="alert alert-warning">
    <strong>Note:</strong> To keep the output clean when dealing with unverified SSL certificates, you can disable the <code>InsecureRequestWarning</code>. This step is not needed for demos, but it helps in simplifying the input representation by avoiding unnecessary warnings.
</div>

In [ ]:
import warnings
import urllib3

# Suppress the specific InsecureRequestWarning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

<p class="lead">
   To streamline our code, we'll define global variables for the username, password, and URLS. We securely collect the user's credentials using the <code>getpass</code> module to ensure the password remains hidden
</p>


In [ ]:
import getpass

# Ask for credentials
username = input("Enter your username: ")
password = getpass.getpass("Enter your password: ")

# Store username and password in a tuple
auth = (username,password)

<p class="lead"><strong>auth</strong>: This variable contains the authentication credentials as a tuple, used for HTTP calls that require authorization.</p>


In [ ]:
# Define URLs for schema creation, data insertion and other utilities
create_global_schema_rest_url = f"{base_url}/rest/template/global/"
create_data_rest_url = f"{base_url}/rest/data/"
run_query_rest_url = f"{base_url}/rest/data/query/"
create_template_html_rendering_rest_url = f"{base_url}/rest/template/html_rendering/"

<h2 class="text-primary">Step 2: Quick Start with a basic Example</h2>
 

<p class="lead">In this section, we provide a simple, complete example: create a schema, insert data, retrieve it, and display the results.</p>

<h3>Define Schema</h3>

<p class="lead">This schema defines a person structure with 6 properties: an <code>id</code> (integer), <code>name</code> (string), <code>age</code>  (integer), <code>address</code> (string), <code>tel</code>(string), and <code>email</code> (string).</p>


In [ ]:
import json
import requests
from IPython.display import display, HTML
from datetime import datetime

# Get the current date and time
current_datetime = datetime.now().strftime("%Y_%m_%d_%H:%M:%S")

# Define the JSON content for the schema creation
json_content = {
        "title": f"schema_{current_datetime}",
        "filename": f"schema_{current_datetime}.json",
        "content": json.dumps({
  "type": "object",
  "properties": {
    "id": { "type": "integer" },
    "name": { "type": "string" },
    "age": { "type": "integer" },
    "address": { "type": "string" },
    "tel": { "type": "string" },
    "email": { "type": "string" },
  }  
    })}
            
# Make a POST request to create the schema
response = requests.post(create_global_schema_rest_url, json=json_content, auth=auth, verify=False)
            
# Check the response status
if response.status_code == 201:
    response_data = response.json()
    display(HTML(f'<div class="alert alert-success">Success: {response_data.get("_display_name")} created</div>'))
    template_id = response_data.get('id')
else:
    display(HTML(f'<div class="alert alert-danger"> Failed: {response.status_code} - {response.text}</div>'))

<h3>Insert the data</h3>

<p class="lead">
  This data defines a JSON object for creating a person entry with fields such as <code>id</code>, <code>name</code>,<code>age</code>, <code>tel</code>,<code>email</code> and <code>address</code>.
</p>


In [ ]:
import json
import requests
from datetime import datetime

# Get the current date and time
current_datetime = datetime.now().strftime("%Y_%m_%d_%H:%M:%S")

# Define the JSON content for the data creation
json_content = {
    "title": f"data_{current_datetime}",
    "template": template_id,  # Ensure template_id is set from previous schema creation
    "content": json.dumps({
        "id": 1,
        "name": "Alice",
        "age": 30,
        "address":"123 Main St, Cityville, State 12345",
        "tel": "+1 (234) 567-890",
        "email": "alice@example.com"
    })
}

# Make a POST request to create the data entry
response = requests.post(create_data_rest_url, json=json_content, auth=auth, verify=False)

# Check the response status
if response.status_code == 201:
    response_data = response.json()
    display(HTML(f'<div class="alert alert-success">Success: {response_data.get("title") } created</div>'))
    data_id = response_data.get('id')  

else:
    display(HTML(f'<div class="alert alert-danger"> Failed: {response.status_code} - {response.text}</div>'))


<p class="lead">Let's retrieve data by sending a query that combines criteria like titles and templates to match the data we created, enabling efficient filtering and sorting of results.</p>

<h3>Retreive data</h3>

<p class="lead">
  To retrieve all data entries, the following POST request is used. The query is executed with <code>"all": "true"</code> to return all available records from the database.
</p>


In [ ]:
response = requests.post(run_query_rest_url, json={"query": {},"title": f"data_{current_datetime}" , "all": "true"}, auth= auth,verify=False )


if response.status_code == 200:
    display(HTML(f'<div class="alert alert-success"> Success <pre>{json.dumps(response.json(), indent=5)}</pre> </div>'))
else:
    display(HTML(f'<div class="alert alert-danger"> Failed: {response.status_code} - {response.text}</div>'))

<h3>Render data via template html rendering</h3> 

<p class="lead">
  We are creating a detailed visual representation of data using a Bootstrap card layout with icons for enhanced clarity. The template dynamically pulls information such as <strong>name</strong>, <strong>age</strong>,<strong> tel</strong>, <strong>address</strong> and <strong>email</strong>, and presents it in a visually styled card format.
</p>



In [ ]:
# Define the HTML rendering template
detail_rendering = """
<iframe srcdoc="
<!DOCTYPE html>
<html lang='en'>
<head>
  <!-- Bootstrap 5 CSS and Icons only for this iframe -->
  <link href='https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha1/dist/css/bootstrap.min.css' rel='stylesheet'>
  <link href='https://cdn.jsdelivr.net/npm/bootstrap-icons/font/bootstrap-icons.css' rel='stylesheet'>
</head>
<body>
<div class='card border-info mb-3' style='max-width: 80%; margin: 20px auto;'>
  <div class='card-header bg-info text-white d-flex align-items-center'>
    <i class='bi bi-person-circle me-2' style='font-size: 3rem;'></i> <!-- Enlarged person icon with custom color -->
    <h4 class='mb-0'>Person Details</h4>
  </div>
  <div class='card-body'>
    <div class='d-flex align-items-center mb-3'>
      <i class='bi bi-person-fill me-2'></i> 
      <strong>Name:</strong> {{ dict_content.name }}
    </div>
    <div class='d-flex align-items-center mb-3'>
      <i class='bi bi-calendar3 me-2'></i> 
      <strong>Age:</strong> {{ dict_content.age }}
    </div>
    <div class='d-flex align-items-center'>
      <i class='bi bi-geo-alt-fill me-2'></i>
      <strong>Address:</strong> {{ dict_content.address }}
    </div>
  </div>
  <ul class='list-group list-group-flush'>
  </ul>
  <div class='card-body'>
    <p class='card-text'>
      <i class='bi bi-telephone me-2'></i> <a href='{{ dict_content.tel }}'>{{ dict_content.tel }}</a><br>
      <i class='bi bi-envelope me-2'></i> <a href='{{ dict_content.email }}'>{{ dict_content.email }}</a>
    </p>
  </div>
</div>
</body>
</html>
" width="100%" height="400px;" style="border:none;"></iframe>
"""
# Prepare the content dictionary
content = {
    "template": template_id,  # Use the previously created template id
    "detail_rendering": detail_rendering 
}

# Make a POST request to create the template HTML rendering
response = requests.post(create_template_html_rendering_rest_url, json=content, auth=auth, verify=False)

# Check the response status
if response.status_code == 201:
    display(HTML(f'<div class="alert alert-success"> Success: HTML rendering created</div>'))  
else:
    display(HTML(f'<div class="alert alert-danger"> Failed: {response.status_code} - {response.text}</div>'))


<p class="lead">
  Here's the result after the transformation. Run the code to see the rendered output:
</p>

In [ ]:
from IPython.display import display, HTML 

# Construct the URL
render_data_detail_rest_url = f"{base_url}/rest/data/{data_id}/render/"

# Make a GET request to fetch the rendered HTML detail for the data entry
response = requests.get(render_data_detail_rest_url, auth=auth, verify=False)

# Check the response status
if response.status_code == 200:
    display(HTML(response.json()))
else:
    display(HTML(f'<div class="alert alert-danger"> Failed: {response.status_code} - {response.text}</div>'))


<div class="alert alert-success"> You’ve completed the quick start with a basic example where you successfully created, inserted,retrieved and displayed your json data. This gave you a simple overview of how to interact with the CDCS platform. </div>

# Exploring Advanced Schema Templates, Data and Queries
 

<p class="lead">Now, it's time to dive deeper into the details. We'll explore the next steps, such as defining more complex schema templates, submitting them to the platform, and running more advanced queries to manipulate and retrieve data. These steps will give you a stronger grasp of the CDCS platform’s capabilities and how to customize it for your specific needs.</p>

<h2 class="text-primary">Step 1: Creating a Template from JSON Schema</h2>

<p class="lead">Let's define the function that creates the global schema and submits it to the specified API endpoint:</p>


In [ ]:
def create_new_schema(title, schema_content):
    # Prepare the content dictionary with title, filename, and schema content
    content = {
        "title": title,  
        "filename": f"{title}.json",  
        "content": json.dumps(schema_content) 
    }

    # Send a POST request to create the schema
    response = requests.post(create_global_schema_rest_url, json=content, auth=auth, verify=False)

    # Check the response status
    if response.status_code == 201:
        display(HTML(f'<div class="alert alert-success"> Success: {title} created</div>'))
        response_data = response.json()  
        return response_data.get('id')  
    else:
        display(HTML(f'<div class="alert alert-danger"> Failed: {response.status_code} - {response.text}</div>'))


<p class="lead">The function takes a schema title and JSON content, then sends a POST request to the API.</p>

<p class="lead">We'll explore three JSON schema types: empty, simple, and complex, each serving different validation and structure needs.</p>

## Empty JSON Schema

<p class="lead">An empty JSON schema <code>{}</code> is very permissive, allowing any valid JSON data to be considered valid. This includes any primitive value, array, object, and complex nested structures.</p>

<div class="alert alert-warning">
    <strong>Note:</strong> In the CDCS application, only JSON objects are considered valid data, so while the schema is permissive, the data must still be a JSON object.
</div>



In [ ]:
# Empty JSON schema
empty_schema = {
    "title": "schema_1",
    "content": "{}" # Empty content
}

# Create the schema_1
schema_id_1 = create_new_schema("schema_1", empty_schema)


<p class="lead">Once the schema is created, it will become available as:</p>
<ul class="lead">
    <li>An option for curating data at <strong>/curate</strong></li>
    <li >A filter option in the exploration section at <strong>/explore/keyword</strong></li>
    <li>An item listed in the templates section at <strong>/core-admin/templates</strong></li>
</ul>

## Simple JSON Schema

<p class="lead">A simple JSON schema defines a basic structure for the data.</p>

<p class="lead">For example, a schema that expects an object Person with specific fields such as <strong>Name</strong> <code>string</code> and <strong>Age</strong>  <code>integer</code> can enforce these constraints, ensuring that only JSON data matching this structure is valid.</p>

In [ ]:
simple_schema_content = {
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "title": "Person",
    "type": "object",
    "properties": {
        "name": {
            "type": "string"
        },
        "age": {
            "type": "integer",
        }
    }
}

# Create the schema_2
schema_id_2 = create_new_schema("schema_2", simple_schema_content)

<p class="lead">If you check <strong>/curate/</strong> again, you'll see two options for curating data: <strong>schema_1</strong> and <strong>schema_2</strong>. Additionally, these schemas will appear as:</p>
<ul class="lead">
    <li>A filter option in the exploration section at <strong>/explore/keyword</strong></li>
    <li>An item listed in the templates section at <strong>/core-admin/templates</strong></li>
</ul>

## Complex JSON Schema

<p class="lead">A complex JSON schema provides a detailed structure for the data, enforcing strict validation rules.</p>

<p class="lead">For example, the schema might expect an object with nested fields such as <strong>name</strong> (<code>string</code>), <strong>age</strong> (<code>integer</code> with a minimum value of 0), and <strong>address</strong> (an object containing <strong>street</strong>, <strong>city</strong>, <strong>state</strong>, and <strong>postalCode</strong>, with <strong>postalCode</strong> requiring a specific pattern). Additionally, the schema can include arrays, such as <strong>hobbies</strong>, where each item must be a <code>string</code>.</p>

<p class="lead">This ensures that only JSON data conforming to this intricate structure is considered valid.</p>



In [ ]:
complex_schema_content = {
  "$id": "https://example.com/complex-object.schema.json",
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "title": "Complex Object",
  "type": "object",
  "properties": {
    "name": {
      "type": "string"
    },
    "age": {
      "type": "integer",
      "minimum": 0
    },
    "address": {
      "type": "object",
      "properties": {
        "street": {
          "type": "string"
        },
        "city": {
          "type": "string"
        },
        "state": {
          "type": "string"
        },
        "postalCode": {
          "type": "string",
          "pattern": "\\d{5}"
        }
      },
      "required": ["street", "city", "state", "postalCode"]
    },
    "hobbies": {
      "type": "array",
      "items": {
        "type": "string"
      }
    }
  },
  "required": ["name", "age"]
}

# Create the schema_3
schema_id_3 = create_new_schema("schema_3", complex_schema_content)

<p class="lead">You can hop over to <strong>/curate/</strong> and check it out. </p>

<div class="alert alert-warning">
    <strong>Note:</strong> If you run one of those codes multiple times with different titles `Schema_X`, you'll start seeing all your options for curating data show up there.
</div>

<h2 class="text-primary">Step 2: Creating data</h2> 

<p class="lead">With our templates defined, we'll now create a function to submit data structured according to these templates via a POST request to the API.</p>

<p class="lead">The <code>create_new_data</code> function is responsible for creating a new data entry. We need to provide a title, the template ID, and the JSON content.</p>

In [ ]:
def create_new_data(title, template, data_content):
    
    # Prepare the content dictionary with title, template, and schema content
    content = {
        "title": title,
        "template": template,
        "content": json.dumps(data_content) 
    }
    
    # Send a POST request to create the schema
    response = requests.post(create_data_rest_url, json=content, auth=auth, verify=False)
    
    # Check the response status
    if response.status_code == 201:
        display(HTML(f'<div class="alert alert-success"> Success: {title} created</div>'))
        response_data = response.json()  
        return response_data.get('id') 
    else:
        display(HTML(f'<div class="alert alert-danger"> Failed: {response.status_code} - {response.text}</div>'))

<p class="lead"> The next code includes different JSON data examples to test the API's ability to handle various dataset using an empty template.</p>



### Creating data using empty schema

<ul class="lead"><li>The first example is an empty JSON object <code>{}</code>, which serves as a minimal input to check how the API handles the absence of data.</li></ul>



In [ ]:
data_content = {}

# Create the data_1 from the schema 1
data_id_1 = create_new_data("data_1_schema_1", schema_id_1, data_content)

<ul class="lead"><li>The second example is a simple object <code>{"key": "value"}</code>, which contains a single key-value pair, representing basic JSON data. This is useful for testing straightforward data submissions.</li></ul>


In [ ]:
data_content = {"key": "value"}

# Create the data_2 from the schema 1
data_id_2 = create_new_data("data_2_schema_1", schema_id_1, data_content)

<ul class="lead">
    <li>The third example is a complex nested JSON object that includes an array and a nested object. This example, with keys like <code>"array"</code>, <code>"object"</code>, and <code>"nested"</code>, is designed to process and store more intricate JSON structures.</li>
</ul>


In [ ]:
data_content = {  # complex nested JSON
        "array": [1, 2, 3],
        "object": {"key": "value"},
        "nested": {
            "arr": [False, True],
            "obj": {"innerKey": 123}
        }
    }

# Create the data_3 from the schema 1
data_id_3 = create_new_data("data_3_schema_1", schema_id_1, data_content)

<ul class="lead">
    <li>Finally, we'll try using a string. As mentioned before, the error occurs because <code>data_content = "12"</code> is a plain string, not a valid JSON object.
    </li>
</ul>


In [ ]:
data_content = "12"

# Attempt to create data with a correct type, but invalid for our system 
create_new_data("data_4_schema_1", schema_id_1, data_content)

<p class="lead">Together, these examples demonstrate how permissive an empty template can be, accommodating everything from the simplest to the most complex JSON.</p>

### Creating data using the simple schema

<p class="lead">When using a simple schema, we define a basic structure for our JSON data, specifying field types. The schema ensures data integrity while still allowing flexibility in the submitted content.</p>

In [ ]:
data_content = {
    "name": "username",
    "age": 30
}

# Create the data_1 from the schema 2
data_id_4 = create_new_data("data_1_schema_2", schema_id_2 , data_content)

<p class="lead">Creating data with an empty string for the name field technically pass validation because an empty string is still considered a valid string type.</p>

In [ ]:
data_content = {
    "name": "",
    "age": 40
}

# Create the data_2 from the schema 2
data_id_5 = create_new_data("data_2_schema_2", schema_id_2 , data_content)

<p class="lead">The next entry includes an extra field <code>email</code> that is not defined in the schema. While the schema doesn't specify this field, it will still be accepted unless the schema explicitly disallows additional properties.</p>

In [ ]:
data_content = {
    "name": "John Smith",
    "age": 25,
    "email": "john.smith@example.com"
}

# Create the data_3 from the schema 2
data_id_6 = create_new_data("data_3_schema_2", schema_id_2 , data_content)

<p class="lead">Creating data with an incorrect type for the <code>age</code> field, such as providing a string (<code>thirty</code>) instead of the expected integer, will violate the schema's type requirement. As a result, the system will reject the entry and return an error, as it doesn't conform to the defined schema.</p>

In [ ]:
data_content = {
    "name": "Jane Doe",
    "age": "thirty"
}

# Attempt to create data with an incorrect type for 'age'
create_new_data("data_schema_2", schema_id_2, data_content)

### Creating data using the complex schema

<p class="lead">Now, we'll create data entries using the complex JSON schema. This schema enforces strict validation rules, ensuring that data is structured and conforms to specific constraints.</p>

In [ ]:
data_content ={
  "name": "John Doe",
  "age": 30,
  "address": {
    "street": "123 Main St",
    "city": "Anytown",
    "state": "CA",
    "postalCode": "12345"
  },
  "hobbies": ["reading", "cycling", "hiking"]
}

# Create the data_1 from the schema 3
data_id_7 = create_new_data("data_1_schema_3", schema_id_3, data_content)

<p class="lead">In the next example, we'll intentionally provide incorrect input to demonstrate schema validation. The <code>age</code> is <code>-5</code> (must be non-negative), the postal code <code>"ABCDE"</code> doesn't match the required pattern, and <code>hobbies</code> are given as a string instead of an array. These errors will trigger validation failures.</p>

In [ ]:
data_content ={
  "name": "John Doe",
  "age": -5,
  "address": {
    "street": "789 Pine Road",
    "city": "Gotham",
    "state": "NY",
    "postalCode": "ABCDE"
  },
  "hobbies": "reading, cycling, hiking"
}

# Attempt to create data with an incorrect values
create_new_data("data_schema_3", schema_id_3, data_content)

<p class="lead">
Now that all your data has been created, you can view it in the <strong>My records</strong> section of the app or manipulate it using queries.</p>

<div class="alert alert-warning">
    <strong>Note:</strong> If you run one of those codes multiple times, you'll see identical data in the list, as creating data with the same name from the same template is allowed.
</div>

<h2 class="text-primary">Step 3: Retreive Data using queries</h2> 

<p class="lead">In this section, we explore different query options for retrieving data from the API. </p>

<p class="lead">Queries can be customized based on specific criteria like pagination, filtering by title, workspaces, templates, or even XPath expressions. You can also combine multiple query options to fine-tune the results, allowing for flexible and efficient data retrieval.</p>

<p class="lead">Before we begin, let's create a function that sends a POST request to execute a specified query on the platform.</p>


In [ ]:
# Function to run a query and handle the response
def run_query(query):
    
    # Send a POST request to the query endpoint with the query data
    response = requests.post(run_query_rest_url, json=query, auth=auth, verify=False)
    
    # Check the response status
    if response.status_code == 200:
        display(HTML(f'<div class="alert alert-success"> Success <pre>{json.dumps(response.json(), indent=5)}</pre> </div>'))
    else:
        display(HTML(f'<div class="alert alert-danger"> Failed: {response.status_code} - {response.text}</div>'))


<ul class="lead"><li> Get all results ensuring all possible matches are returned:</li></ul>

In [ ]:
run_query({"query": {}, "all": "true"})

<ul class="lead"><li>  Get all results filtered by a specific title:</li></ul>

In [ ]:
run_query({"query": {}, "title": "data_3_schema_2", "all": "true"})

<ul class="lead"><li>  Get results filtered by a specific templates and keywords:</li></ul>

In [ ]:
run_query({"query": {"name":"John Smith","age":25}, "templates": [{"id": schema_id_2},{"id": schema_id_3}], "all": "true"})

<h2 class="text-primary">Step 4: Template HTML Rendering</h2>   

<p class="lead">Template HTML rendering allows you to dynamically display data in either a list or detail by defining and adjusting specific templates for these formats.</p>

<p class="lead">To explore this feature, we'll create a template HTML rendering for each of our templates to see how our data is rendered in each option.</p>


In [ ]:
create_template_html_rendering_rest_url = f"{base_url}/rest/template/html_rendering/"
render_data_list_rest_url = f"{base_url}/rest/data/{data_id_7}/render?rendering=list"
render_data_detail_rest_url = f"{base_url}/rest/data/{data_id_7}/render/"

<p class="lead">
    The <code>create_new_template_html_rendering</code> function facilitates the creation of custom HTML templates for displaying data associated with a specific schema.It takes the template ID, list rendering HTML, and detail rendering HTML as inputs.
.</p>


In [ ]:
# Function to create a new HTML rendering for a template
def create_new_template_html_rendering(template):
    # Prepare the content dictionary with the template, list rendering, and detail rendering
    content = {
        "template": template,  
    }

    # Send a POST request to create the template HTML rendering on the server
    response = requests.post(create_template_html_rendering_rest_url, json=content, auth=auth, verify=False)

    # Check the response status
    if response.status_code == 201:
        display(HTML('<div class="alert alert-success"> Success: HTML rendering created</div>'))
        response_data = response.json()  
        return response_data.get('id') 
    else:
        display(HTML(f'<div class="alert alert-danger"> Failed: {response.status_code} - {response.text}</div>'))


template_html_rendering_id = create_new_template_html_rendering(schema_id_3)

<div class="alert alert-warning">
    <strong>Note:</strong> Every template should be associated with only one template_html_rendering. If you try to run this function more than once for the same template, it will result in an error.
</div>

<p class="lead">
    To avoid errors from creating multiple renderings for the same template, we will create the initial <code>template_html_rendering</code> and update the list and detail rendering each time you need to make changes.
</p>


In [ ]:
from IPython.display import display, HTML

# Function to update a HTML rendering for a template
def update_rendering(template_rendering, list_rendering, detail_rendering):
    # Prepare the content dictionary with the id, list rendering, and detail rendering
    content = {
        "list_rendering": list_rendering,
        "detail_rendering": detail_rendering,
    }
    
    create_template_html_rendering_rest_url = f"{base_url}/rest/template/html_rendering/{template_rendering}"

    # Send a patch request to update the template HTML rendering on the server
    response = requests.patch(create_template_html_rendering_rest_url, json=content, auth=auth, verify=False)

    # Check the response status
    if response.status_code == 200:
        display(HTML('<div class="alert alert-success"> Success: HTML rendering updated</div>'))
       
    else:
        display(HTML(f'<div class="alert alert-danger"> Failed: {response.status_code} - {response.text}</div>'))

In [ ]:
list_rendering = list_rendering = """<div class="table-responsive">
  <table class="table table-striped">
    <tbody>
      <tr>
        <th style="width: 30%;">Name</th>
        <td style="width: 70%;">{{ dict_content.name }}</td>
      </tr>
      <tr>
        <th style="width: 30%;">Age</th>
        <td style="width: 70%;">{{ dict_content.age }}</td>
      </tr>
      <tr>
        <th style="width: 30%;">Address</th>
        <td style="width: 70%;">{{ dict_content.address.street }}, {{ dict_content.address.city }}, {{ dict_content.address.state }} {{ dict_content.address.postalCode }}</td>
      </tr>
    </tbody>
  </table>
</div>"""



detail_rendering = """<div class="container">
  <h2>Details</h2>
  <table class="table table-striped" style="width: 80%; table-layout: fixed;">
    <tbody>
      <tr>
        <th style="width: 30%;">Name</th>
        <td style="width: 70%;">{{ dict_content.name }}</td>
      </tr>
      <tr>
        <th style="width: 30%;">Age</th>
        <td style="width: 70%;">{{ dict_content.age }}</td>
      </tr>
    </tbody>
  </table>

  <h3>Address</h3>
  <table class="table table-striped" style="width: 80%; table-layout: fixed;">
    <tbody>
      <tr>
        <th style="width: 30%">Street</th>
        <td style="width: 70%;" class="float-right">{{ dict_content.address.street }}</td>
      </tr>
      <tr>
        <th style="width: 30%;">City</th>
        <td style="width: 70%;">{{ dict_content.address.city }}</td>
      </tr>
      <tr>
        <th style="width: 30%;">State</th>
        <td style="width: 70%;">{{ dict_content.address.state }}</td>
      </tr>
      <tr>
        <th style="width: 30%;">Postal Code</th>
        <td style="width: 70%;">{{ dict_content.address.postalCode }}</td>
      </tr>
    </tbody>
  </table>

  <h3>Hobbies</h3>
  <ul class="list-group list-group-flush" style="width: 80%;">
    {% for hobby in dict_content.hobbies %}
      <li class="list-group-item">{{ hobby }}</li>
    {% endfor %}
  </ul>
</div>"""



update_rendering(template_html_rendering_id, list_rendering, detail_rendering)

<p class="lead">To view how the data appears with the rendering applied, use the <code>get_template_html_rendering</code> function. After creating the <code>template_html_rendering</code>, retrieve and display the HTML-rendered content by calling the function with the appropriate URL for either the list or detail format.</p>


In [ ]:
from IPython.display import display, HTML

def get_template_html_rendering(render_data_url):
    response = requests.get(render_data_url,auth= auth,verify=False)
    if response.status_code == 200:
        display(HTML(response.json()))
    else:
        print(f"Failed: {response.status_code} - {response.text}")

In [ ]:
get_template_html_rendering(render_data_list_rest_url)

<p class="lead">
  The result displays the details of a specific data entry using a clean, well-organized layout, utilizing the list HTML rendering associated with the template.
</p>


In [ ]:
get_template_html_rendering(render_data_detail_rest_url)

<p class="lead">
  The result displays the details of a specific data entry using a clean, well-organized layout, utilizing the detail HTML rendering associated with the template.
</p>
